<a href="https://www.kaggle.com/code/tay208/test-on-polypgen-dataset?scriptVersionId=300061432" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
%cd /kaggle/input
DatasetRoot = "/kaggle/input/datasets/kokoroou/polypgen2021/PolypGen2021_MultiCenterData_v3/positive"
import os

directory_path = "/kaggle/input/datasets/kokoroou/polypgen2021/PolypGen2021_MultiCenterData_v3/positive" # Example path

if os.path.isdir(directory_path):
    print(f"The directory exists: {directory_path}")
else:
    print(f"The directory does not exist: {directory_path}")


/kaggle/input
The directory exists: /kaggle/input/datasets/kokoroou/polypgen2021/PolypGen2021_MultiCenterData_v3/positive


In [2]:
import os
from PIL import Image
import torch.utils.data as data
import torchvision.transforms as transforms
import numpy as np
import random
import torch
class test_dataset:
    def __init__(self, image_root, gt_root, testsize):
        self.testsize = testsize
        
        # 1. Get all files and use os.path.join for safety
        # We use .lower() to handle .JPG vs .jpg issues
        self.images = [os.path.join(image_root, f) for f in os.listdir(image_root) 
                       if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
        
        self.gts = [os.path.join(gt_root, f) for f in os.listdir(gt_root) 
                    if f.lower().endswith(('.tif', '.png', '.jpg', '.jpeg'))]
        
        # 2. Sort both lists so they match perfectly
        self.images = sorted(self.images)
        self.gts = sorted(self.gts)
        
        # 3. Validation print - check your output!
        self.size = len(self.images)
        print(f"--- Dataset Loaded ---")
        print(f"Found {self.size} images and {len(self.gts)} masks.")
        
        if self.size != len(self.gts):
            print("WARNING: Mismatch between image count and mask count!")
            # Fallback: only use the minimum count to avoid IndexError
            self.size = min(self.size, len(self.gts))

        self.transform = transforms.Compose([
            transforms.Resize((self.testsize, self.testsize)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
        
        self.gt_transform = transforms.ToTensor()
        self.index = 0

    def load_data(self):
        # Prevent the "Index out of range" crash
        if self.index >= self.size:
            print("Reached end of dataset, resetting index.")
            self.index = 0
            
        image = self.rgb_loader(self.images[self.index])
        image = self.transform(image).unsqueeze(0)
        gt = self.binary_loader(self.gts[self.index])
        
        # Get filename without the path
        name = os.path.basename(self.images[self.index])
        # if name.endswith('.jpg'):
        #     name = name.replace('.jpg', '.png')
        name = os.path.basename(self.images[self.index])
        self.index += 1
        return image, gt, name

    def rgb_loader(self, path):
        with Image.open(path) as img:
            return img.convert('RGB')

    def binary_loader(self, path):
        with Image.open(path) as img:
            return img.convert('L')

In [3]:
import sys
sys.path.append('/kaggle/input/polyppvt/Polyp-PVT')
import torch
import torch.nn.functional as F
import numpy as np
import os, argparse
from scipy import misc
from lib.pvt import PolypPVT
# from utils.dataloader import test_dataset
import cv2
import time
start_time = time.time()
%cd /kaggle/input/polyppvt/Polyp-PVT
parser = argparse.ArgumentParser()
parser.add_argument('--testsize', type=int, default=352, help='testing size')
parser.add_argument('--pth_path', type=str, default='./model_pth/PolypPVT.pth')
opt = parser.parse_args(args=[])
model = PolypPVT()
model.load_state_dict(torch.load(opt.pth_path))
model.cuda()
model.eval()

# data_path = '/kaggle/input/PolypGen2021/PolypGen2021_MultiCenterData_v3/positive/TestDataset/{}'.format(_data_name)
data_path = DatasetRoot 
##### save_path #####
save_path = '/kaggle/working/result_map/{}/'.format("positive")

if not os.path.exists(save_path):
    os.makedirs(save_path)
image_root = '{}/images/'.format(data_path)
gt_root = '{}/masks/'.format(data_path)
num1 = len(os.listdir(gt_root))
test_loader = test_dataset(image_root, gt_root, 352)

print(num1) 
print(f"Total Images: {len(test_loader.images)}")
print(f"Total Masks: {len(test_loader.gts)}")
print(f"Current Index: {test_loader.index}")
for i in range(num1):
    image, gt, name = test_loader.load_data()
    gt = np.asarray(gt, np.float32)
    gt /= (gt.max() + 1e-8)
    image = image.cuda()
    P1,P2 = model(image)
    res = F.upsample(P1+P2, size=gt.shape, mode='bilinear', align_corners=False)
    res = res.sigmoid().data.cpu().numpy().squeeze()
    res = (res - res.min()) / (res.max() - res.min() + 1e-8)
    cv2.imwrite(save_path+name, res*255)
end_time = time.time() - start_time

print('Finish!')
print(f"Execution time: {end_time} seconds")


/tmp/ipykernel_23/2429616362.py:7: DeprecationWarning: scipy.misc is deprecated and will be removed in 2.0.0
  from scipy import misc
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to

/kaggle/input/polyppvt/Polyp-PVT
--- Dataset Loaded ---
Found 3762 images and 3762 masks.
3762
Total Images: 3762
Total Masks: 3762
Current Index: 0


/kaggle/input/polyppvt/Polyp-PVT/lib/pvt.py:91: UserWarning: `nn.functional.upsample` is deprecated. Use `nn.functional.interpolate` instead.
  edge = F.upsample(edge, (x.size()[-2], x.size()[-1]))
/tmp/ipykernel_23/2429616362.py:45: UserWarning: `nn.functional.upsample` is deprecated. Use `nn.functional.interpolate` instead.
  res = F.upsample(P1+P2, size=gt.shape, mode='bilinear', align_corners=False)
[ WARN:0@6.301] global loadsave.cpp:1063 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.


Finish!
Execution time: 370.86092591285706 seconds
